# ETL Project

## State Level Household Debt Statistics 2003-2018

Source: https://www.newyorkfed.org/medialibrary/interactives/householdcredit/data/xls/area_report_by_year.xlsx

This data source provides composition of household debt per state per year in the United States. Idea is to extract, transform, and load this data to determine if there is correlation between amount of debt per household and complains reported to the *Consumer Financial Protection Bureau (CFBP)* within the same state and the same period of time. 

First, let's get the resource into a local file within the `Resources` folder.

In [1]:
import pandas as pd
import urllib.request

url = 'https://www.newyorkfed.org/medialibrary/interactives/householdcredit/data/xls/area_report_by_year.xlsx'
filename = 'Resources/area_report_by_year.xlsx'
urllib.request.urlretrieve(url, filename)

('Resources/area_report_by_year.xlsx',
 <http.client.HTTPMessage at 0x11868de80>)

Now, let's import the first sheet –`auto`– in the local file.

In [10]:
df = pd.read_excel(filename, 'auto', header=0, skiprows=3)
df.set_index('state', inplace=True)
df.head()

,Q4_2003,Q4_2004,Q4_2005,Q4_2006,Q4_2007,Q4_2008,Q4_2009,Q4_2010,Q4_2011,Q4_2012,Q4_2013,Q4_2014,Q4_2015,Q4_2016,Q4_2017,Q4_2018
state,,,,,,,,,,,,,,,,
AK,3480,3390,3590,3620,3800,3780,3620,3680,3790,4140,4270,4690,4990,5070,5010.0,4920.0
AL,2910,2970,3100,3320,3540,3520,3170,3130,3220,3480,3590,3910,4200,4500,4780.0,4960.0
AR,3070,2960,3190,3280,3400,3400,3190,3140,3310,3640,3830,4200,4640,4970,5240.0,5380.0
AZ,3420,3800,4030,4220,4350,4190,3610,3290,3330,3540,3710,4050,4480,4750,4890.0,5200.0
CA,3150,3300,3530,3640,3550,3450,3020,2770,2770,2950,3130,3420,3820,4140,4380.0,4580.0


Remove `Q4_` from column headers

In [11]:
new_column_headers = []
for column_header in df.columns:
    new_column_headers.append(column_header.replace('Q4_', ''))
df.columns = new_column_headers
df.head()

,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
state,,,,,,,,,,,,,,,,
AK,3480,3390,3590,3620,3800,3780,3620,3680,3790,4140,4270,4690,4990,5070,5010.0,4920.0
AL,2910,2970,3100,3320,3540,3520,3170,3130,3220,3480,3590,3910,4200,4500,4780.0,4960.0
AR,3070,2960,3190,3280,3400,3400,3190,3140,3310,3640,3830,4200,4640,4970,5240.0,5380.0
AZ,3420,3800,4030,4220,4350,4190,3610,3290,3330,3540,3710,4050,4480,4750,4890.0,5200.0
CA,3150,3300,3530,3640,3550,3450,3020,2770,2770,2950,3130,3420,3820,4140,4380.0,4580.0


Normalize this table, so we have one column indicating the year.

In [12]:
new_df = None
for column in df.columns:
    df_year = df[column].to_frame()
    df_year['year'] = column
    df_year.rename(columns={column: 'auto'}, inplace=True)
    if new_df is None:
        new_df = df_year
    else:
        new_df = pd.concat([new_df, df_year])
new_df.reset_index(inplace=True)
new_df.set_index(['year', 'state'], inplace=True)
new_df.head()
debt_df = new_df
debt_df.head()

auto
year state        
2003 AK     3480.0
     AL     2910.0
     AR     3070.0
     AZ     3420.0
     CA     3150.0

Now we established the process to do the Extract/Transform for the `auto` table, we can change it to iterate over all the sheets within the source.

In [31]:
new_df = None
debt_df = None

sheets = ['auto', 'creditcard', 'mortgage', 'studentloan', 'total', 
          'auto_delinq', 'creditcard_delinq', 'mortgage_delinq', 'studentloan_delinq', 'population']

for sheet in sheets:
    df = pd.read_excel(filename, sheet, header=0, skiprows=3)
    df.set_index('state', inplace=True)
    
    new_column_headers = []
    for column_header in df.columns:
        new_column_headers.append(column_header.replace('Q4_', ''))
    df.columns = new_column_headers
    df.head()

    new_df = None
    for column in df.columns:
        df_year = df[column].to_frame()
        df_year['year'] = int(column)
        df_year.rename(columns={column: sheet}, inplace=True)
        if new_df is None:
            new_df = df_year
        else:
            new_df = pd.concat([new_df, df_year])
    new_df.reset_index(inplace=True)
    new_df.set_index(['year', 'state'], inplace=True)
    if debt_df is None:
        debt_df = new_df
    else:
        debt_df = debt_df.merge(new_df, left_index=True, right_index=True, how='inner')
debt_df.head()

auto  creditcard  mortgage  studentloan    total  auto_delinq  \
year state                                                                    
2003 AK     3480.0      4260.0   26080.0        680.0  36420.0         0.83   
     AL     2910.0      2600.0   14750.0        880.0  24390.0         2.63   
     AR     3070.0      2430.0   11880.0        710.0  20260.0         2.06   
     AZ     3420.0      3110.0   27360.0       1080.0  37720.0         2.86   
     CA     3150.0      3080.0   39410.0        970.0  49410.0         1.89   

            creditcard_delinq  mortgage_delinq  studentloan_delinq  population  
year state                                                                      
2003 AK                  5.21             0.57                5.50    478640.0  
     AL                 11.92             1.57                8.20   3780480.0  
     AR                 11.22             1.32                5.79   2140020.0  
     AZ                  8.97             1.05                7.44   4280840.0  
     CA                  8.84             0.43                6.55  27970460.0

In [32]:
debt_df.count()

auto                  846
creditcard            846
mortgage              846
studentloan           846
total                 846
auto_delinq           846
creditcard_delinq     846
mortgage_delinq       846
studentloan_delinq    830
population            847
dtype: int64

Now, time to load the results into the database.

In [26]:
import config as cfg
from sqlalchemy import create_engine

rds_connection_string = f"{cfg.postgresql['user']}:{cfg.postgresql['passwd']}@" \
                        f"{cfg.postgresql['host']}:{str(cfg.postgresql['port'])}/" \
                        f"{cfg.postgresql['db']}"
rds_connection_string

'postgres:postgres@localhost:5432/etl_project'

In [28]:
engine = create_engine(f'postgresql+pg8000://{rds_connection_string}')
engine.table_names()

['consumer_debt']

In [34]:
debt_df.to_sql(name='consumer_debt', con=engine, if_exists='append', index=True)